# Query Snowflake in Python

In [ ]:
# copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk
%env snowflakeuser=<your_snowflake_username>
%env snowflakepass=<your_snowflake_password>

In [2]:
# Packages
import snowflake.connector
import pandas as pd
import os
import numpy as np

In [3]:
def snowflake_to_pandas(connection_params, query):
    try:
        # Establish a connection to Snowflake
        conn = snowflake.connector.connect(**connection_params)

        # Execute the SQL query and fetch the results into a DataFrame
        df = pd.read_sql_query(query, conn)

        # Close the connection
        conn.close()

        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# SQL command 
query = 'SELECT * FROM "ANALYTICS"."FORECASTING"."sales_fcast_combined_v"'

# Call the function to retrieve the data into a Pandas DataFrame
result_df = snowflake_to_pandas(connection_params, query)

if result_df is not None:
    print(result_df.head())  # Display the first few rows of the DataFrame
else:
    print("Failed to retrieve data from Snowflake.")

c:\Users\crudek\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   DEP_ENT       MONTH   NET_SALES  BUDGET_AMOUNT  FORECAST
0  240_155  2017-02-01  4153796.71      4198000.0       NaN
1  240_155  2017-03-01  5940741.71      4892400.0       NaN
2  240_155  2017-04-01  4789757.73      5580200.0       NaN
3  240_155  2017-05-01  6356822.97      5298000.0       NaN
4  240_155  2017-06-01  5756533.29      4678000.0       NaN


# Phase 1
Lets understand the packages and process being used and forecast for one department-entity combination before doing multiple series

Referecen: https://moez-62905.medium.com/time-series-in-python-10502f9fac2a

In [4]:
# head and tail
print(result_df)

      DEP_ENT       MONTH    NET_SALES  BUDGET_AMOUNT  FORECAST
0     240_155  2017-02-01   4153796.71      4198000.0       NaN
1     240_155  2017-03-01   5940741.71      4892400.0       NaN
2     240_155  2017-04-01   4789757.73      5580200.0       NaN
3     240_155  2017-05-01   6356822.97      5298000.0       NaN
4     240_155  2017-06-01   5756533.29      4678000.0       NaN
...       ...         ...          ...            ...       ...
1260  210_155  2016-07-01   6283148.80     11311652.0       NaN
1261  210_155  2016-08-01   5626725.62     11326527.0       NaN
1262  210_155  2016-09-01  10283086.45     11377763.0       NaN
1263  210_155  2016-10-01   6239512.66     11377763.0       NaN
1264  210_155  2016-11-01  16467966.49     11690688.0       NaN

[1265 rows x 5 columns]


In [5]:
result_df["MONTH"] = pd.to_datetime(result_df["MONTH"])
print(result_df.head())
print(result_df.dtypes)

   DEP_ENT      MONTH   NET_SALES  BUDGET_AMOUNT  FORECAST
0  240_155 2017-02-01  4153796.71      4198000.0       NaN
1  240_155 2017-03-01  5940741.71      4892400.0       NaN
2  240_155 2017-04-01  4789757.73      5580200.0       NaN
3  240_155 2017-05-01  6356822.97      5298000.0       NaN
4  240_155 2017-06-01  5756533.29      4678000.0       NaN
DEP_ENT                  object
MONTH            datetime64[ns]
NET_SALES               float64
BUDGET_AMOUNT           float64
FORECAST                float64
dtype: object


In [6]:
df_200_155 = result_df[result_df["DEP_ENT"] == '200_155']
df_200_155 = df_200_155[df_200_155['MONTH'] <= pd.Timestamp(2023,7,1)] # dont have data beyond july '23 so no way to check model accuracy
df_200_155 = df_200_155[["MONTH", "NET_SALES"]]
print(df_200_155)
print(df_200_155.dtypes)

         MONTH   NET_SALES
192 2016-01-01  2592017.04
193 2016-02-01  2348092.91
194 2016-03-01  2894584.98
195 2016-04-01  2163636.39
196 2016-05-01  3210147.13
..         ...         ...
358 2023-03-01  2325654.24
359 2023-04-01  1918474.98
360 2023-05-01  2443940.14
361 2023-06-01  1748262.30
362 2023-07-01  1787253.79

[91 rows x 2 columns]
MONTH        datetime64[ns]
NET_SALES           float64
dtype: object


In [7]:
# ML Can't deal with dates directly so we need to extract some basic properties

# extract month and year from dates
df_200_155['Month'] = [i.month for i in df_200_155['MONTH']]
df_200_155['Year'] = [i.year for i in df_200_155['MONTH']]
# create a sequence of numbers
df_200_155['Series'] = np.arange(1,len(df_200_155)+1)
print(df_200_155.head())

         MONTH   NET_SALES  Month  Year  Series
192 2016-01-01  2592017.04      1  2016       1
193 2016-02-01  2348092.91      2  2016       2
194 2016-03-01  2894584.98      3  2016       3
195 2016-04-01  2163636.39      4  2016       4
196 2016-05-01  3210147.13      5  2016       5


In [8]:
# Create training dataset
train = df_200_155[df_200_155['MONTH'] <= pd.Timestamp(2022,4,1)] # ~80%
# drop unnecessary columns and re-arrange
train.drop(['MONTH'], axis=1, inplace=True)
train = train[['Series', 'Year', 'Month', 'NET_SALES']] 


# Create testing dataset
test = df_200_155[df_200_155['MONTH'] > pd.Timestamp(2022,4,1)] # ~20%
# drop unnecessary columns and re-arrange
test.drop(['MONTH'], axis=1, inplace=True)
test = test[['Series', 'Year', 'Month', 'NET_SALES']] 

C:\Users\crudek\AppData\Local\Temp\ipykernel_18372\3638390786.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['MONTH'], axis=1, inplace=True)
C:\Users\crudek\AppData\Local\Temp\ipykernel_18372\3638390786.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(['MONTH'], axis=1, inplace=True)


In [9]:
# good if around 80/20 split train/test
train.shape, test.shape

((76, 4), (15, 4))

In [11]:
# import the regression module
from pycaret.regression import *

In [12]:
# initialize setup
s = setup(data = train, test_data = test, target = 'NET_SALES', 
          fold_strategy = 'timeseries', numeric_features = ['Year', 'Series'], 
          fold = 3, transform_target = True, session_id = 123)

,Description,Value
0,session_id,123
1,Target,NET_SALES
2,Original Data,"(76, 4)"
3,Missing Values,False
4,Numeric Features,2
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(76, 14)"


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'

In [13]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,668399.0208,1030151187114.6666,936187.0625,0.2708,0.3107,0.2293,0.0233
omp,Orthogonal Matching Pursuit,711189.7146,1140784284289.5813,991312.9469,0.1736,0.3470,0.2298,0.0100
et,Extra Trees Regressor,779254.1833,1265486693020.8792,1047946.5743,0.0824,0.3654,0.2714,0.0433
br,Bayesian Ridge,784503.9877,1461220371853.1116,1138730.8473,-0.0639,0.3978,0.2314,0.0100
rf,Random Forest Regressor,791551.1630,1264045637190.9138,1059010.9908,0.0482,0.3589,0.2886,0.0467
ada,AdaBoost Regressor,854444.2589,1422919758483.0898,1117075.3035,-0.0849,0.3909,0.3170,0.0200
lightgbm,Light Gradient Boosting Machine,859050.5261,1607270538713.0977,1237842.0621,-0.2908,0.4014,0.2692,0.2467
huber,Huber Regressor,862025.1889,1579071834084.3113,1200356.2640,-0.1852,0.4592,0.2681,0.0133
lr,Linear Regression,872798.3333,1616293582165.3333,1195575.8125,-0.2049,0.4557,0.2678,1.2967
catboost,CatBoost Regressor,873321.1890,1495809178779.3984,1178721.1134,-0.2065,0.3700,0.2997,0.3067
